In [1]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 1

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0.1])

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 9


100%|██████████| 320/320 [00:14<00:00, 21.98it/s]


(0.6014240768051284,
 0.4185750000000001,
 0.54575,
 0.34354810870108954,
 0.3646140582325625)

In [3]:
res_truth1 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0
)
res_truth2 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.3,
    R=100,
    burnin_R=100,
    seed=0
)

ground_truth1 = {}
ground_truth1['average'] = res_truth1['average']
ground_truth1['direct'] = res_truth1['direct_effect']
ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
ground_truth1['psi_zero'] = res_truth1['psi_zero']

ground_truth1

100%|██████████| 200/200 [00:06<00:00, 32.71it/s]


{'average': 0.6585375,
 'direct': 0.3125749999999999,
 'indirect': 0.014400000000000135,
 'spillover_effect': 0.023600000000000232,
 'psi_1_gamma': 0.75475,
 'psi_0_gamma': 0.44217500000000015,
 'psi_zero': 0.4185749999999999}

In [ ]:
from utils import run_pll
from run_pll import run_dr_raw

treat_p = 0.7
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': False,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

Multiprocessing <function run_dr_raw at 0x17fd04dc0> in 100 tasks, with 10 processes...


In [ ]:
from utils import run_pll
from run_pll import run_dr_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': False,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

Multiprocessing <function run_dr_raw at 0x14d3acdc0> in 100 tasks, with 10 processes...
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/run_pll.py", line 49, in run_dr_raw
    ret_i = doubly_robust(A_chain[i], L_chain[i], Y_chain[i], adj, treatment_allocation=treatment_allocation, seed=1, return_raw=True,
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py", line 198, in doubly_robust
    denominator = get_norm_constant_new(A, GL, neighbours, neighbours_2hop, gamma, adj_matrix)
TypeError: get_norm_constant_new() missing 1 required positional argument: 'adj_matrix'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/utils.py", line 17, in log_error
    raise error

In [ ]:
# read results
import numpy as np
from utils import *


results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

0it [00:00, ?it/s]
/opt/anaconda3/envs/ljz/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/ljz/lib/python3.10/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/opt/anaconda3/envs/ljz/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/envs/ljz/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/envs/ljz/lib/python3.10/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([nan, nan, nan, nan, nan, nan, nan]),
 'bias': array([nan, nan, nan, nan, nan, nan, nan]),
 'mse': array([nan, nan, nan, nan, nan, nan, nan]),
 'var': array([nan, nan, nan, nan, nan, nan, nan]),
 'ci_length': array([nan, nan, nan, nan, nan, nan, nan]),
 'true_effect': array([0.6585375, 0.312575 , 0.0144   , 0.0236   , 0.75475  , 0.442175 ,
        0.418575 ])}

In [ ]:
from tqdm import tqdm
from drnet import compute_avg_effects_std_from_raw

results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)

var = []
var_sim = []
est = []
idx = 1
for i in tqdm(range(Y_chain.shape[0])):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(results1[i,:,idx],adj1,h=3)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(results1[i,:,idx]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

  0%|          | 0/100 [00:00<?, ?it/s]


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed